<a href="https://colab.research.google.com/github/RusAl84/text_mining/blob/master/27_02_2022_%D0%9C%D0%B0%D1%81%D1%82%D0%B5%D1%80_%D0%BA%D0%BB%D0%B0%D1%81%D1%81_%D0%92%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B2_%D0%B8%D0%BD%D1%82%D0%B5%D0%BB%D0%BB%D0%B5%D0%BA%D1%82%D1%83%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TF-IDF

У подхода bag-of-words есть существенный недостаток. Если слово встречается 5 раз в конкретном документе, но и в других документах тоже встречается часто, то его наличие в документе не особо-то о чём-то говорит. Если же слово 5 раз встречается в конкретном документе, но в других документах встречается редко, то его наличие (да ещё и многократное) позволяет хорошо отличать этот документ от других. Однако с точки зрения bag-of-words различий не будет: в обеих ячейках будет просто число 5.

Отчасти это решается исключением стоп-слов (и слишком часто встречающихся слов), но лишь отчасти. Другой идеей является отмасштабировать получившуюся таблицу с учётом "редкости" слова в наборе документов (т.е. с учётом информативности слова).

tfidf=tf∗idf

idf=log((N+1)/(Nw+1))+1

Здесь tf это частота слова в тексте (то же самое, что в bag of words), N - общее число документов, Nw - число документов, содержащих данное слово.

То есть для каждого слова считается отношение общего количества документов к количеству документов, содержащих данное слово (для частых слов оно будет ближе к 1, для редких слов оно будет стремиться к числу, равному количеству документов), и на логарифм от этого числа умножается исходное значение bag-of-words (к числителю и знаменателю прибавляется единичка, чтобы не делить на 0, и к логарифму тоже прибавляется единичка, но это уже технические детали). После этого в sklearn ещё проводится L2-нормализация каждой строки.

В sklearn есть класс для поддержки TF-IDF: TfidfVectorizer, рассмотрим его.

In [1]:
texts = "\u0422\u043E \u0435\u0441\u0442\u044C \u0434\u043B\u044F \u043A\u0430\u0436\u0434\u043E\u0433\u043E \u0441\u043B\u043E\u0432\u0430 \u0441\u0447\u0438\u0442\u0430\u0435\u0442\u0441\u044F \u043E\u0442\u043D\u043E\u0448\u0435\u043D\u0438\u0435 \u043E\u0431\u0449\u0435\u0433\u043E \u043A\u043E\u043B\u0438\u0447\u0435\u0441\u0442\u0432\u0430 \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u043E\u0432 \u043A \u043A\u043E\u043B\u0438\u0447\u0435\u0441\u0442\u0432\u0443 \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u043E\u0432, \u0441\u043E\u0434\u0435\u0440\u0436\u0430\u0449\u0438\u0445 \u0434\u0430\u043D\u043D\u043E\u0435 \u0441\u043B\u043E\u0432\u043E (\u0434\u043B\u044F \u0447\u0430\u0441\u0442\u044B\u0445 \u0441\u043B\u043E\u0432 \u043E\u043D\u043E \u0431\u0443\u0434\u0435\u0442 \u0431\u043B\u0438\u0436\u0435 \u043A 1, \u0434\u043B\u044F \u0440\u0435\u0434\u043A\u0438\u0445 \u0441\u043B\u043E\u0432 \u043E\u043D\u043E \u0431\u0443\u0434\u0435\u0442 \u0441\u0442\u0440\u0435\u043C\u0438\u0442\u044C\u0441\u044F \u043A \u0447\u0438\u0441\u043B\u0443, \u0440\u0430\u0432\u043D\u043E\u043C\u0443 \u043A\u043E\u043B\u0438\u0447\u0435\u0441\u0442\u0432\u0443 \u0434\u043E\u043A\u0443\u043C\u0435\u043D\u0442\u043E\u0432), \u0438 \u043D\u0430 \u043B\u043E\u0433\u0430\u0440\u0438\u0444\u043C \u043E\u0442 \u044D\u0442\u043E\u0433\u043E \u0447\u0438\u0441\u043B\u0430 \u0443\u043C\u043D\u043E\u0436\u0430\u0435\u0442\u0441\u044F \u0438\u0441\u0445\u043E\u0434\u043D\u043E\u0435 \u0437\u043D\u0430\u0447\u0435\u043D\u0438\u0435 bag-of-words (\u043A \u0447\u0438\u0441\u043B\u0438\u0442\u0435\u043B\u044E \u0438 \u0437\u043D\u0430\u043C\u0435\u043D\u0430\u0442\u0435\u043B\u044E \u043F\u0440\u0438\u0431\u0430\u0432\u043B\u044F\u0435\u0442\u0441\u044F \u0435\u0434\u0438\u043D\u0438\u0447\u043A\u0430, \u0447\u0442\u043E\u0431\u044B \u043D\u0435 \u0434\u0435\u043B\u0438\u0442\u044C \u043D\u0430 0, \u0438 \u043A \u043B\u043E\u0433\u0430\u0440\u0438\u0444\u043C\u0443 \u0442\u043E\u0436\u0435 \u043F\u0440\u0438\u0431\u0430\u0432\u043B\u044F\u0435\u0442\u0441\u044F \u0435\u0434\u0438\u043D\u0438\u0447\u043A\u0430, \u043D\u043E \u044D\u0442\u043E \u0443\u0436\u0435 \u0442\u0435\u0445\u043D\u0438\u0447\u0435\u0441\u043A\u0438\u0435 \u0434\u0435\u0442\u0430\u043B\u0438). \u041F\u043E\u0441\u043B\u0435 \u044D\u0442\u043E\u0433\u043E \u0432 sklearn \u0435\u0449\u0451 \u043F\u0440\u043E\u0432\u043E\u0434\u0438\u0442\u0441\u044F L2-\u043D\u043E\u0440\u043C\u0430\u043B\u0438\u0437\u0430\u0446\u0438\u044F \u043A\u0430\u0436\u0434\u043E\u0439 \u0441\u0442\u0440\u043E\u043A\u0438." #@param {type:"string"}
stexts = texts
print(texts)

То есть для каждого слова считается отношение общего количества документов к количеству документов, содержащих данное слово (для частых слов оно будет ближе к 1, для редких слов оно будет стремиться к числу, равному количеству документов), и на логарифм от этого числа умножается исходное значение bag-of-words (к числителю и знаменателю прибавляется единичка, чтобы не делить на 0, и к логарифму тоже прибавляется единичка, но это уже технические детали). После этого в sklearn ещё проводится L2-нормализация каждой строки.


In [2]:
# !pip install scikit-learn
!pip install pandas==1.2.3
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [3]:
def printTF_IDF(texts):
  records_count = 30
  tfIdfTransformer = TfidfVectorizer(ngram_range=(1, 4), use_idf=True, max_features=records_count)
  countVectorizer = CountVectorizer(ngram_range=(1, 4), max_features=records_count)
  wordCount = countVectorizer.fit_transform([texts])
  TfIdf = tfIdfTransformer.fit_transform([texts])
  names = countVectorizer.get_feature_names()
  df=[]
  df = pd.DataFrame(list(names), columns=['names'])
  df = df.assign(Word_Count=wordCount.T.todense())
  df = df.assign(TF_IDF=TfIdf.T.todense())
  df = df.sort_values('TF_IDF', ascending=False)
  print(df)

In [4]:
printTF_IDF(texts)

                                names  Word_Count    TF_IDF
2                          документов           3  0.325396
1                                 для           3  0.325396
0                               будет           2  0.216930
9                        прибавляется           2  0.216930
22                     слов оно будет           2  0.216930
21                           слов оно           2  0.216930
20                               слов           2  0.216930
10              прибавляется единичка           2  0.216930
29                              этого           2  0.216930
8                           оно будет           2  0.216930
4                          количеству           2  0.216930
6                                  на           2  0.216930
3                            единичка           2  0.216930
7                                 оно           2  0.216930
5               количеству документов           2  0.216930
28              слово для частых слов   

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


удалим знаки переноса строк и сделаем текст в нижнем регистре

In [22]:
  str2 = ''
  for item in texts.split():
    str2 = str2 + ' ' + item
  texts = str(texts)
  texts = texts.lower()
  # print(texts)
  texts = texts.replace('\n', ' ')
  print(texts)

 каждого слова считается отношение общего количества документов количеству документов содержащих данное слово частых слов оно ближе редких слов оно стремиться числу равному количеству документов логарифм числа умножается исходное значение bag of words числителю знаменателю прибавляется единичка делить логарифму прибавляется единичка это технические детали sklearn ещё проводится нормализация каждой строки


удалим цифры

In [23]:
  str2 = ''
  for c in texts:
      if c not in ('0', "1", '2', '3', '4', '5', '6', '7', '8', '9', '«', '»', '–', "\""):
          str2 = str2 + c
  texts = str2
  str2 = ''
  print(texts)

 каждого слова считается отношение общего количества документов количеству документов содержащих данное слово частых слов оно ближе редких слов оно стремиться числу равному количеству документов логарифм числа умножается исходное значение bag of words числителю знаменателю прибавляется единичка делить логарифму прибавляется единичка это технические детали sklearn ещё проводится нормализация каждой строки


удалим знаки пунктуации

In [7]:
import string
pattern = string.punctuation
for c in texts:
    if c not in pattern:
        str2 = str2 + c
    else:
        str2 = str2 + " "
texts = str2
str2 = ''

удалим стоп слова

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
  from nltk.corpus import stopwords
  russian_stopwords = stopwords.words("russian")
  for word in texts.split():
      if word not in (russian_stopwords):
          str2 = str2 + " " + word
  texts = str2

  str2 = ''
  for word in texts.split():
      if len(word) > 1:
          str2 = str2 + " " + word
  texts = str2
  print(texts)

 каждого слова считается отношение общего количества документов количеству документов содержащих данное слово частых слов оно ближе редких слов оно стремиться числу равному количеству документов логарифм числа умножается исходное значение bag of words числителю знаменателю прибавляется единичка делить логарифму прибавляется единичка это технические детали sklearn ещё проводится нормализация каждой строки


рассчитаем TF-IDF

In [10]:
printTF_IDF(texts)

                                      names  Word_Count  TF_IDF
1                                документов           3   0.381
9                              прибавляется           2   0.254
2                                  единичка           2   0.254
3                                количеству           2   0.254
4                     количеству документов           2   0.254
7                                       оно           2   0.254
22                                     слов           2   0.254
23                                 слов оно           2   0.254
10                    прибавляется единичка           2   0.254
24                           слов оно ближе           1   0.127
21               редких слов оно стремиться           1   0.127
0                                       bag           1   0.127
25                    слов оно ближе редких           1   0.127
19                              редких слов           1   0.127
26                      слов оно стремит

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


#Суммаризации текстов 

Задача суммаризации текстов (автореферирование) - одна из ключевых, широко обсуждаемых задач NLP. Она состоит в сжатии больших объемов текста до связного краткого содержания, отражающего только основные идеи.


#Алгоритм TextRank
TextRank - это алгоритм, основанный на PageRank, который часто используется для извлечения ключевых слов и суммирования текста. 
PageRank (PR) - это алгоритм, используемый для расчета веса веб-страниц. 


In [11]:
from gensim.summarization import summarize, keywords
print("Исходный текст:")
print(stexts)
print("Результат работы TextRank:")
print(summarize(str(stexts)))

Исходный текст:
То есть для каждого слова считается отношение общего количества документов к количеству документов, содержащих данное слово (для частых слов оно будет ближе к 1, для редких слов оно будет стремиться к числу, равному количеству документов), и на логарифм от этого числа умножается исходное значение bag-of-words (к числителю и знаменателю прибавляется единичка, чтобы не делить на 0, и к логарифму тоже прибавляется единичка, но это уже технические детали). После этого в sklearn ещё проводится L2-нормализация каждой строки.
Результат работы TextRank:



#Алгоритм Rake
RAKE: Rapid Automatic Keyword Extraction Algorithm
Алгоритм RAKE извлекает ключевые слова с помощью основанного на разделителе подхода, чтобы идентифицировать ключевые слова кандидата и баллы их использующий совместные встречаемости слова, которые появляются в ключевых словах кандидата. Ключевые слова могут содержать несколько лексем. Кроме того, алгоритм RAKE также объединяет ключевые слова, когда они кажутся многократно, разделенными тем же разделителем слияния.


In [12]:
!pip install rake_nltk
from rake_nltk import Metric, Rake
r = Rake(language="russian")
import nltk
nltk.download('punkt')
r.extract_keywords_from_text(stexts)
mas = r.get_ranked_phrases()
set2 = set()
for item in mas:
    if not "nan" in str(item).replace(" nan ", " "):
        set2.add(str(item).replace(" nan ", " "))
mas = list(set2)
print("Исходный текст:")
print(stexts)
print("Результат работы TextRammk:")
print(str(mas))

     |████████████████████████████████| 1.5 MB 4.8 MB/s 
     |████████████████████████████████| 748 kB 37.3 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Исходный текст:
То есть для каждого слова считается отношение общего количества документов к количеству документов, содержащих данное слово (для частых слов оно будет ближе к 1, для редких слов оно будет стремиться к числу, равному количеству документов), и на логарифм от этого числа умножается исходное значение bag-of-words (к числителю и знаменателю прибавляется единичка, чтобы не делить на 0, и к логарифму тоже прибавляется единичка, но это уже технические детали). После этого в sklearn ещё проводится L2-нормализация каждой строки.
Результат работы TextRammk:
['sklearn ещё проводится l2', '1', 'частых слов оно', 'числителю', '0', 'words', 'знаменателю прибавляется единичка', 'числу', 'это', 'каждого слова считается отношение общего количества документов', 'нормализация каждой строки', 'редких слов оно', 'of', 'прибавляется единичка', 'ближе', 'технические детали ).', 'числа умно

#Тематическое моделирование
Тематическое моделирование — это метод извлечения тем из текста. Latent Dirichlet Allocation (LDA) — популярный алгоритм моделирования тем реализованные в том числе в пакете Gensim. Основная задача алгоритмов ТМ, заключается в том что бы полученные темы были хорошего качество, понятными, самозначимыми и разделенными. Достижение этих целей во многом зависит от качества предварительной обработки текста и стратегии поиска оптимального количества тем. 

##Импорт пакетов


In [13]:
import nltk; nltk.download('stopwords')
import re
import numpy as np
import pandas as pd
from pprint import pprint
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# Plotting tools
import matplotlib.pyplot as plt
%matplotlib inline
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Подход LDA к тематическому моделированию заключается в том, что каждый документ рассматривается как набор тем в определенной пропорции. И каждая тема как набор ключевых слов, опять же, в определенной пропорции.

После того, как вы укажете алгоритму количество тем, все, что он сделает, — это отобразит распределение тем в документах и распределение ключевых слов по темам.

Тема — это не что иное, как набор доминирующих ключевых слов. Просто взглянув на ключевые слова, вы сможете определить, о чем эта тема.

Ниже приведены ключевые факторы для получения хороших разделительных тем:

Качество обработки текста.
Разнообразие тем, о которых говорится в тексте.
Выбор алгоритма моделирование тем.
Количество тем, указанных в алгоритме.
Алгоритмы настройки параметров.

# Подготовим стоп-слова


In [14]:
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

##Лемматизация
Лемматизация — это не что иное, как преобразование слова в его корневое слово. Например: лемма слова «machines» — это «machine». Аналогично, «walking» -> «walk», «mice» -> «mouse» и так далее.

In [15]:
!pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
ltexts = ""
print(texts)
for word in texts.split():
    if len(str(word)) > 2:
        ltexts+=" " + morph.parse(word)[0].normal_form
print(ltexts)

     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 8.2 MB 8.5 MB/s 
 каждого слова считается отношение общего количества документов количеству документов содержащих данное слово частых слов оно ближе редких слов оно стремиться числу равному количеству документов логарифм числа умножается исходное значение bag of words числителю знаменателю прибавляется единичка делить логарифму прибавляется единичка это технические детали sklearn ещё проводится нормализация каждой строки
 каждый слово считаться отношение общий количество документ количество документ содержать данный слово частый слово оно близкий редкий слово оно стремиться число равный количество документ логарифм число умножаться исходный значение bag words числитель знаменатель прибавляться единичка делить логарифм прибавляться единичка это технический деталь sklearn ещё проводиться нормализация каждый строка


##Создадим словарь и корпус.
Двумя основными входными данными для тематической модели LDA являются словарь (id2word) и корпус. Давайте создадим их.

In [16]:
from nltk.util import ngrams
def make_bigrams(ttexts):
    texts = ' '.join(ttexts)
    token = nltk.word_tokenize(ttexts)
    bigrams = list(ngrams(token, 2))
    # print(bigrams)
    return bigrams
def make_trigrams(ttexts):
    texts = ' '.join(ttexts)
    token = nltk.word_tokenize(ttexts)
    trigrams = list(ngrams(token, 2))
    return trigrams

btexts = make_bigrams(texts) + make_trigrams(texts)
id2word = corpora.Dictionary(btexts)
corpus = [id2word.doc2bow(text) for text in btexts]
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('каждого', 1), ('слова', 1)]]

Или вы можете увидеть удобочитаемую форму самого корпуса.

In [17]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('каждого', 1), ('слова', 1)]]

##Построим тематическую модель
У нас есть все необходимое для обучения модели LDA. В дополнение к корпусу и словарю необходимо также указать количество тем.

In [18]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

##Просмотр тем в модели LDA
Вышеупомянутая модель LDA построена на 20 различных темах, где каждая тема представляет собой комбинацию ключевых слов, и каждое ключевое слово вносит определенный вес в тему.

Вы можете увидеть ключевые слова для каждой темы и вес (важность) каждого ключевого слова, используя lda_model.print_topics().

In [19]:
lda_model.print_topics()

[(0,
  '0.116*"слов" + 0.116*"оно" + 0.115*"логарифм" + 0.115*"документов" + 0.115*"количества" + 0.060*"каждой" + 0.060*"нормализация" + 0.060*"числа" + 0.060*"общего" + 0.006*"sklearn"'),
 (1,
  '0.156*"единичка" + 0.118*"прибавляется" + 0.080*"проводится" + 0.080*"исходное" + 0.080*"ещё" + 0.042*"of" + 0.042*"значение" + 0.042*"делить" + 0.042*"знаменателю" + 0.042*"words"'),
 (2,
  '0.104*"bag" + 0.104*"редких" + 0.104*"частых" + 0.104*"слов" + 0.055*"of" + 0.055*"значение" + 0.054*"прибавляется" + 0.054*"строки" + 0.054*"каждой" + 0.054*"слово"'),
 (3,
  '0.080*"равному" + 0.080*"числителю" + 0.080*"числу" + 0.080*"оно" + 0.080*"стремиться" + 0.080*"детали" + 0.080*"технические" + 0.042*"логарифму" + 0.042*"ближе" + 0.042*"sklearn"'),
 (4,
  '0.156*"документов" + 0.118*"количеству" + 0.080*"слова" + 0.080*"данное" + 0.080*"считается" + 0.080*"отношение" + 0.080*"содержащих" + 0.042*"слово" + 0.042*"числа" + 0.042*"умножается"')]

##Визуализация темы и ключевых слов
Теперь, когда модель LDA создана, следующим шагом является изучение созданных тем и связанных с ними ключевых слов. Нет лучшего инструмента, чем интерактивная диаграмма пакета pyLDAvis.

In [20]:
!pip install pyLDAvis==2.1.2
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

     |████████████████████████████████| 1.6 MB 4.4 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97738 sha256=7763ae9c216d2581a3a1f08b6ea4d28723c95c8227f8142d8d7b2cd751992ddc
  Stored in directory: /root/.cache/pip/wheels/3b/fb/41/e32e5312da9f440d34c4eff0d2207b46dc9332a7b931ef1e89
Successfully built pyLDAvis


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [21]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.106961 -0.180802       1        1  22.671934
1     -0.184314 -0.001597       2        1  22.655564
4      0.228425 -0.062956       3        1  22.618924
2     -0.062045  0.192021       4        1  16.900551
0      0.124895  0.053335       5        1  15.153026, topic_info=            Term       Freq      Total Category  logprob  loglift
6     документов  10.000000  10.000000  Default  30.0000  30.0000
30      единичка   7.000000   7.000000  Default  29.0000  29.0000
12          слов   7.000000   7.000000  Default  28.0000  28.0000
13           оно   7.000000   7.000000  Default  27.0000  27.0000
19      логарифм   4.000000   4.000000  Default  26.0000  26.0000
..           ...        ...        ...      ...      ...      ...
35        детали   0.160039   4.138822   Topic5  -5.2029  -1.3658
7     количеству   0.160141   7.450181   Topic5  -5.2022  -1.9530
30      единичка   0.160110   7.453211   Topic5  -5.2024  -1.9536
29  прибавляется   0.160091   7.404435   Topic5  -5.2025  -1.9471
33           это   0.160031   4.137403   Topic5  -5.2029  -1.3655

[217 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
24        4  0.742994        bag
25        2  0.489372         of
25        4  0.489372         of
36        1  0.483426    sklearn
36        2  0.483426    sklearn
...     ...       ...        ...
20        5  0.490775      числа
27        1  0.724842  числителю
17        1  0.724842      числу
33        1  0.483395        это
33        2  0.483395        это

[62 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 5, 3, 1])